In [ ]:
pip install requests

In [ ]:
pip install pandas

In [1]:
import requests

In [12]:
import pandas as pd

In [3]:
class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key

    def fetch_intraday_data(self, script):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=1min&apikey={self.api_key}"
        response = requests.get(url)
        data = response.json()
        self.data = data["Time Series (1min)"]
        
    def convert_intraday_data(self, script):
        timestamps = []
        opens = []
        highs = []
        lows = []
        closes = []
        volumes = []

        for timestamp, values in self.data.items():
            timestamps.append(pd.Timestamp(timestamp))
            opens.append(float(values["1. open"]))
            highs.append(float(values["2. high"]))
            lows.append(float(values["3. low"]))
            closes.append(float(values["4. close"]))
            volumes.append(int(values["5. volume"]))

        df = pd.DataFrame({
            "timestamp": timestamps,
            "open": opens,
            "high": highs,
            "low": lows,
            "close": closes,
            "volume": volumes
        })

        return df

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __contains__(self, key):
        return key in self.data
    
    def __contains__(self, script):
        return script in self.data

In [4]:
def indicator1(df, timeperiod):
    indicator = df['close'].rolling(window=timeperiod).mean()
    result = pd.DataFrame({'timestamp': df['timestamp'], 'indicator': indicator})
    return result

In [5]:
class Strategy:
    def __init__(self, script):
        self.script = script
        self.data = None
        self.indicator_data = None
        self.signals = None

    def fetch_intraday_data(self, api_key):
        script_data = ScriptData(api_key)
        script_data.fetch_intraday_data(self.script)
        self.data = script_data.convert_intraday_data(self.script)

    def compute_indicator_data(self, timeperiod):
        self.indicator_data = indicator1(self.data, timeperiod)

    def generate_signals(self):
        if self.data is None or self.indicator_data is None:
            raise ValueError("Data and indicator data must be computed first.")

        close_data = self.data['close']
        indicator_data = self.indicator_data['indicator']
        signals = pd.DataFrame({'timestamp': self.data['timestamp'], 'signal': 'NO_SIGNAL'})

        signals.loc[indicator_data > close_data, 'signal'] = 'BUY'
        signals.loc[indicator_data < close_data, 'signal'] = 'SELL'

        self.signals = signals

    def print_signals(self):
        if self.signals is None:
            raise ValueError("Signals must be generated first.")

        filtered_signals = self.signals[self.signals['signal'].isin(['BUY', 'SELL'])]
        print(filtered_signals)

In [6]:
api_key = '***************'

In [7]:
script = 'GOOGL'

In [8]:
timeperiod = 5

In [9]:
script_data = ScriptData(api_key)

In [10]:
script_data.fetch_intraday_data(script)

In [13]:
df = script_data.convert_intraday_data(script)
df

,timestamp,open,high,low,close,volume
0,2023-06-16 19:59:00,123.650,123.650,123.650,123.650,192
1,2023-06-16 19:58:00,123.600,123.600,123.600,123.600,377
2,2023-06-16 19:57:00,123.630,123.630,123.600,123.600,950
3,2023-06-16 19:53:00,123.710,123.710,123.700,123.700,611
4,2023-06-16 19:51:00,123.750,123.760,123.750,123.760,684
...,...,...,...,...,...,...
95,2023-06-16 15:56:00,123.355,123.495,123.280,123.490,331855
96,2023-06-16 15:55:00,123.560,123.560,123.280,123.355,456467
97,2023-06-16 15:54:00,123.670,123.730,123.540,123.559,265567
98,2023-06-16 15:53:00,123.840,123.895,123.660,123.660,260970


In [14]:
indicator1(df,timeperiod)

,timestamp,indicator
0,2023-06-16 19:59:00,NaN
1,2023-06-16 19:58:00,NaN
2,2023-06-16 19:57:00,NaN
3,2023-06-16 19:53:00,NaN
4,2023-06-16 19:51:00,123.6620
...,...,...
95,2023-06-16 15:56:00,123.5700
96,2023-06-16 15:55:00,123.5010
97,2023-06-16 15:54:00,123.5068
98,2023-06-16 15:53:00,123.5288


In [15]:
strategy = Strategy(script)

In [16]:
strategy.fetch_intraday_data(api_key)

In [17]:
strategy.compute_indicator_data(timeperiod)

In [18]:
strategy.generate_signals()

In [19]:
strategy.print_signals()

             timestamp signal
4  2023-06-16 19:51:00   SELL
5  2023-06-16 19:44:00   SELL
6  2023-06-16 19:40:00   SELL
7  2023-06-16 19:39:00   SELL
8  2023-06-16 19:34:00    BUY
..                 ...    ...
95 2023-06-16 15:56:00    BUY
96 2023-06-16 15:55:00    BUY
97 2023-06-16 15:54:00   SELL
98 2023-06-16 15:53:00   SELL
99 2023-06-16 15:52:00   SELL

[96 rows x 2 columns]
